In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
from matplotlib import pyplot as plt
import re

In [2]:
df = pd.read_csv("datasets/earLobe.csv")
df.head()

,patientID,has_DM2,Var2,Var3,Var4,Var5,Var6,Var7,Var8,Var9,...,Var3152,Var3153,Var3154,Var3155,Var3156,Var3157,Var3158,Var3159,Var3160,Var3161
0,ramanShift,NaN,0.000000,1.000000,2.000000,3.000000,4.000000,5.000000,6.000000,7.000000,...,3150,3151,3152,3153,3154,3155,3156,3157,3158,3159
1,DM201,1.0,181.800000,181.800000,181.800000,181.800000,181.800000,181.800000,181.800000,181.800000,...,0,0,0,0,0,0,0,0,0,0
2,DM202,1.0,162.800000,162.800000,162.800000,162.800000,162.800000,162.800000,162.800000,162.800000,...,0,0,0,0,0,0,0,0,0,0
3,DM203,1.0,107.400000,107.400000,107.400000,107.400000,107.400000,107.400000,107.400000,107.400000,...,0,0,0,0,0,0,0,0,0,0
4,DM204,1.0,290.166667,290.166667,290.166667,290.166667,290.166667,290.166667,290.166667,290.166667,...,0,0,0,0,0,0,0,0,0,0


In [3]:
has_DM2 = df.pop('has_DM2')
patientID = df.pop('patientID')

In [4]:
df.head()

,Var2,Var3,Var4,Var5,Var6,Var7,Var8,Var9,Var10,Var11,...,Var3152,Var3153,Var3154,Var3155,Var3156,Var3157,Var3158,Var3159,Var3160,Var3161
0,0.000000,1.000000,2.000000,3.000000,4.000000,5.000000,6.000000,7.000000,8.000000,9.000000,...,3150,3151,3152,3153,3154,3155,3156,3157,3158,3159
1,181.800000,181.800000,181.800000,181.800000,181.800000,181.800000,181.800000,181.800000,181.800000,181.800000,...,0,0,0,0,0,0,0,0,0,0
2,162.800000,162.800000,162.800000,162.800000,162.800000,162.800000,162.800000,162.800000,162.800000,162.800000,...,0,0,0,0,0,0,0,0,0,0
3,107.400000,107.400000,107.400000,107.400000,107.400000,107.400000,107.400000,107.400000,107.400000,107.400000,...,0,0,0,0,0,0,0,0,0,0
4,290.166667,290.166667,290.166667,290.166667,290.166667,290.166667,290.166667,290.166667,290.166667,290.166667,...,0,0,0,0,0,0,0,0,0,0


In [5]:
%matplotlib widget
plt.plot(df.iloc[0], df.iloc[1]);

FigureCanvasNbAgg()

#### Crop Data to 200-3000 cm^-1 

In [6]:
droped_columns = []
for col in df.columns:
    if int(re.findall(r'\d+', col)[0]) <= 200 or int(re.findall(r'\d+', col)[0]) >= 3000:
        droped_columns.append(col)

In [7]:
df.drop(droped_columns, axis=1, inplace=True)

In [8]:
df.head()

,Var201,Var202,Var203,Var204,Var205,Var206,Var207,Var208,Var209,Var210,...,Var2990,Var2991,Var2992,Var2993,Var2994,Var2995,Var2996,Var2997,Var2998,Var2999
0,199.000000,200.000000,201.000000,202.000000,203.000000,204.000000,205.000000,206.000000,207.000000,208.000000,...,2988.000000,2989.000000,2990.000000,2991.000000,2992.000000,2993.000000,2994.000000,2995.000000,2996.000000,2997.000000
1,201.383380,269.913437,338.443494,358.309815,357.612488,357.066028,357.608710,358.151392,357.265051,355.170612,...,8.836629,6.277010,1.295275,1.000000,12.970419,-0.162721,1.784965,10.615159,0.715051,4.629650
2,179.130545,236.277649,293.424737,313.885785,318.844537,322.219055,314.156335,306.093622,302.820349,303.596069,...,4.553121,2.965374,4.881890,13.139419,8.212818,26.863908,29.267496,28.620376,10.057526,13.393549
3,119.902387,163.653213,207.404037,221.979492,224.226428,226.030222,224.634753,223.239282,223.524774,225.231357,...,6.104761,1.650141,4.399213,16.421537,-0.158094,17.882836,13.442509,10.005216,2.536457,10.222650
4,306.652110,364.341258,422.030406,443.342351,449.282516,454.272054,452.398514,450.524963,451.032817,453.553892,...,6.560440,0.396009,9.085960,8.374695,1.018077,18.895465,21.119795,19.181473,7.916948,14.858042


In [9]:
%matplotlib widget
plt.plot(df.iloc[0], df.iloc[1])

FigureCanvasNbAgg()

In [10]:
list(df.index)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]

In [11]:
list(range(1,21))

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]

In [12]:
columns = list(df.columns)
for row in range(1,21):
    for n,value in enumerate(columns):
        if df[columns[n]][row] < 0 or df[columns[n]][row] > 500:
            df[columns[n]][row] = df[columns[n-1]][row]

In [13]:
%matplotlib widget
plt.plot(df.iloc[0], df.iloc[10])

FigureCanvasNbAgg()

Разделяем данные на матрицу с признаками X и на столбец с целевой переменной (метками)
И удаляемпервую строку с частотами

In [14]:
X, y = df.drop(0), has_DM2.drop(0)